In [116]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [117]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
import pandas as pd
import numpy as np

In [118]:
noOfPages= 2


sentences=[] 

for page in range(1,noOfPages+1):
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print("URL used for scrapping:",url)
    currpage=requests.get(url)
    
    time.sleep(2)   
    soup=BeautifulSoup(currpage.text,'html.parser')
    
    
    listItems =soup.find_all('li',attrs={'class':'o-listicle__item'})

    for j in listItems:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        #Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        sentences.append((Statement))

data=pd.DataFrame(sentences, columns=['Statement'])
data.to_csv(r'NEWS.csv', index = False, header=True)
print(data.head())

URL used for scrapping: https://www.politifact.com/factchecks/list/?page=1
URL used for scrapping: https://www.politifact.com/factchecks/list/?page=2
                                           Statement
0  “Prairie State power plant is the cleanest coa...
1  A flag expressing support for police officers ...
2  "The Second Amendment, from the day it was pas...
3  Says the U.S. government caused the coronaviru...
4  Says Ivanka Trump was targeted in the Florida ...


In [122]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

pattern = 'NP: {<DT>?<JJ>*<NN>}'
parser = nltk.RegexpParser(pattern)


output = pd.DataFrame()

i = 0
for sentence in data['Statement']:
    sent = preprocess(data['Statement'][i])
    parseSent = parser.parse(sent)
    ner_tagged = tree2conlltags(parseSent)
    
    output =output.append(ner_tagged,ignore_index=True)
    i=i+1
    


In [124]:
print("Data Scrapped and POS tagged: \n")
output.head(10)

Data Scrapped and POS tagged: 



,0,1,2
0,“,JJ,O
1,Prairie,NNP,O
2,State,NNP,O
3,power,NN,B-NP
4,plant,NN,B-NP
5,is,VBZ,O
6,the,DT,O
7,cleanest,JJS,O
8,coal-fired,JJ,B-NP
9,power,NN,I-NP


In [125]:
np.savetxt(r'C:\Users\srush\Downloads\NLP\Code2\np.txt', output.values, fmt='%s')